One time run thing:

In [0]:
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType
from pyspark.sql.utils import AnalysisException

from datetime import datetime

from atproto import Client
from atproto_client.models import AppBskyFeedGetTimeline

from textblob import TextBlob

import mlflow
import mlflow.sklearn

# -------------------------------------------------------------------
# 1) Config and paths
# -------------------------------------------------------------------
spark.conf.set(
    "fs.azure.account.key.toxicitylake7032.dfs.core.windows.net",
    "u thought i was about push a key hell nah"
)

BRONZE_PATH = "abfss://lakehouse@toxicitylake7032.dfs.core.windows.net/bronze_live/bluesky/"
SILVER_PATH = "abfss://lakehouse@toxicitylake7032.dfs.core.windows.net/silver_live/bluesky/"
GOLD_PATH   = "abfss://lakehouse@toxicitylake7032.dfs.core.windows.net/gold_live/bluesky_features/"
SCORED_PATH = "abfss://lakehouse@toxicitylake7032.dfs.core.windows.net/gold_live/bluesky_scored/"

LIMIT = 50  # micro-batch size

# -------------------------------------------------------------------
# 2) Load list of already-scored post_ids (for dedup)
# -------------------------------------------------------------------
try:
    df_scored_existing = spark.read.parquet(SCORED_PATH)
    df_existing_ids = df_scored_existing.select("post_id").distinct()
    print(f"Loaded {df_existing_ids.count()} already-scored post_ids.")
except AnalysisException:
    # First run: scored table doesn't exist yet
    df_existing_ids = None
    print("No existing scored data found; this is the first run.")

# -------------------------------------------------------------------
# 3) Login to Bluesky and fetch latest posts
# -------------------------------------------------------------------
bsky_id = dbutils.secrets.get("bsky-scope", "BSKY_ID").strip()
bsky_pw = dbutils.secrets.get("bsky-scope", "BSKY_APP_PASSWORD").strip()

client = Client()
profile = client.login(bsky_id, bsky_pw)
print("Logged in as:", profile.handle)

params = AppBskyFeedGetTimeline.Params(limit=LIMIT)
timeline = client.app.bsky.feed.get_timeline(params=params)

try:
    feed_items = timeline["feed"]
except TypeError:
    feed_items = timeline.feed

print("Items in feed:", len(feed_items))

# -------------------------------------------------------------------
# 4) Build a DataFrame for THIS batch only
# -------------------------------------------------------------------
rows = []

for item in feed_items:
    post = item["post"] if isinstance(item, dict) else item.post
    record = post["record"] if isinstance(post, dict) else post.record
    author = post["author"] if isinstance(post, dict) else post.author

    if isinstance(record, dict):
        text = record.get("text", "")
        created_at = record.get("createdAt")
    else:
        text = getattr(record, "text", "")
        created_at = getattr(record, "created_at", None)

    if isinstance(created_at, str):
        ts = created_at
    elif created_at is not None:
        ts = created_at.isoformat()
    else:
        ts = datetime.utcnow().isoformat()

    if isinstance(author, dict):
        did = author.get("did")
        handle = author.get("handle")
    else:
        did = getattr(author, "did", None)
        handle = getattr(author, "handle", None)

    uri = post["uri"] if isinstance(post, dict) else post.uri

    rows.append(Row(
        post_id = uri,
        user_id = did,
        platform = "bluesky",
        community = "timeline",
        interaction_type = "post",
        publish_ts = ts,
        content = text,
        url = f"https://bsky.app/profile/{handle}/post/{uri.split('/')[-1]}"
    ))

print("Rows built in this batch:", len(rows))

if not rows:
    print("No posts to process.")
    raise SystemExit

df_bronze_batch = spark.createDataFrame(rows)

# -------------------------------------------------------------------
# 5) Remove posts we've already scored (dedup on post_id)
# -------------------------------------------------------------------
if df_existing_ids is not None:
    df_bronze_batch = df_bronze_batch.join(
        df_existing_ids,
        on="post_id",
        how="left_anti"  # keep only post_ids NOT in existing_ids
    )

new_count = df_bronze_batch.count()
print("New, unseen posts in this batch:", new_count)

if new_count == 0:
    print("All posts in this batch were already scored. Nothing new to add.")
    raise SystemExit

# -------------------------------------------------------------------
# 6) Append this batch to Bronze
# -------------------------------------------------------------------
(df_bronze_batch
 .write
 .mode("append")
 .format("json")
 .save(BRONZE_PATH)
)

# -------------------------------------------------------------------
# 7) Create Silver JUST for this batch and append
# -------------------------------------------------------------------
df_silver_batch = (
    df_bronze_batch
    .filter(F.col("content").isNotNull() & (F.length(F.col("content")) > 0))
    .withColumn("publish_ts", F.to_timestamp("publish_ts"))
    .withColumn("publish_date_only", F.to_date("publish_ts"))
)

(df_silver_batch
 .write
 .mode("append")
 .format("parquet")
 .save(SILVER_PATH)
)

# -------------------------------------------------------------------
# 8) Gold features for THIS batch and append
# -------------------------------------------------------------------
def tb_polarity(text):
    if text is None:
        return 0.0
    try:
        return float(TextBlob(text).sentiment.polarity)
    except Exception:
        return 0.0

def tb_subjectivity(text):
    if text is None:
        return 0.0
    try:
        return float(TextBlob(text).sentiment.subjectivity)
    except Exception:
        return 0.0

udf_sentiment_tb = F.udf(tb_polarity)
udf_subjectivity_tb = F.udf(tb_subjectivity)

df_gold_batch = (
    df_silver_batch
    .withColumn("content_length_chars", F.length(F.col("content")))
    .withColumn("content_length_words", F.size(F.split(F.col("content"), r"\s+")))
    .withColumn("sentiment_textblob", udf_sentiment_tb(F.col("content")))
    .withColumn("subjectivity_textblob", udf_subjectivity_tb(F.col("content")))
    .withColumn("sentiment_vader", F.col("sentiment_textblob"))  # proxy
)

(df_gold_batch
 .write
 .mode("append")
 .format("parquet")
 .save(GOLD_PATH)
)

print("Gold batch rows:", df_gold_batch.count())

# -------------------------------------------------------------------
# 9) Score ONLY this Gold batch with ML model
# -------------------------------------------------------------------
pdf_live = df_gold_batch.toPandas()
print("Batch size going into model:", pdf_live.shape)

feature_cols = [
    "content",
    "content_length_words",
    "content_length_chars",
    "sentiment_textblob",
    "subjectivity_textblob",
    "sentiment_vader",
]
X_live = pdf_live[feature_cols]

RUN_ID = "aint no way"
logged_model_uri = f"runs:/{RUN_ID}/model"
print("Loading model from:", logged_model_uri)
reddit_toxicity_model = mlflow.sklearn.load_model(logged_model_uri)

y_pred = reddit_toxicity_model.predict(X_live)
try:
    y_proba = reddit_toxicity_model.predict_proba(X_live)[:, 1]
except Exception:
    y_proba = None

pdf_live["predicted_toxicity_label"] = y_pred
if y_proba is not None:
    pdf_live["predicted_toxicity_score"] = y_proba

df_scored_batch = spark.createDataFrame(pdf_live)

print("Scored batch rows:", df_scored_batch.count())
display(df_scored_batch.select(
    "post_id", "publish_ts", "content",
    "predicted_toxicity_label",
    F.col("predicted_toxicity_score")
).limit(10))

# -------------------------------------------------------------------
# 10) Append scored batch to SCORED_PATH (growing history for Power BI)
# -------------------------------------------------------------------
(df_scored_batch
 .write
 .mode("append")
 .format("parquet")
 .save(SCORED_PATH)
)

print("Appended scored batch to:", SCORED_PATH)


Loaded 140 already-scored post_ids.
Logged in as: [REDACTED]
Items in feed: 50
Rows built in this batch: 50
New, unseen posts in this batch: 6
Gold batch rows: 6
Batch size going into model: (6, 14)
Loading model from: runs:/a0d07d615b264f39ad0a8a2a91a06e7f/model


Scored batch rows: 6


post_id,publish_ts,content,predicted_toxicity_label,predicted_toxicity_score
at://did:plc:77mqbgde4w34rhbkpiqxivbn/app.bsky.feed.post/3m6lry32q2s2s,2025-11-27T06:54:30.272Z,we live and we learn my beautiful boy,0,0.11821968829848106
at://did:plc:iu4j537hox5huj4bwnwgub4z/app.bsky.feed.post/3m6lrf5pga22g,2025-11-27T06:43:55.512Z,"Carney: We know that this decades-long process of our ever-closer economic relationship between Canada and the US has ended. And as a consequence of that, many of our strengths have become our vulnerabilities, particularly in those industries that are most tightly integrated with the US….",0,0.051901279093097656
at://did:plc:s6j27rxb3ic2rxw73ixgqv2p/app.bsky.feed.post/3m6lny4q2fs2n,2025-11-27T05:42:57.05Z,"Dear foreign nationals in war-torn countries, terrorism hot-spots, and amidst our enemies: If you help the United States, we’ll fuck you over and abandon you. Not the individual servicepeople or intelligence officers — they’ll fight for you. I mean the depraved, soulless leadership of the nation.",1,0.5695839678998685
at://did:plc:nefzr6cyllchhg7pevedzate/app.bsky.feed.post/3m6lurkjyb22o,2025-11-27T07:44:32.844Z,I suspect it is the only book he has read,0,0.1099802048448292
at://did:plc:eppsmw5nfmdzhfeymtzd3xbe/app.bsky.feed.post/3m6lmozwwks2d,2025-11-27T05:19:58.399Z,"No idea why an Afghan refugee shot National Guard soldiers within walking distance of the White House, but this new USCIS directive was revealed on Monday. Worth noting.",0,0.12864407744280018
at://did:plc:qw262fidqqagjm4foisra4zp/app.bsky.feed.post/3m6loiuktoc2x,2025-11-27T05:52:18.917Z,"""a single person of this demographic may have committed a crime, therefore we must punish the entire demographic"" is a policy response you may recognize from fascist governments dedicated to ethnic cleansing elsewhere.",0,0.24973993911643716


Appended scored batch to: abfss://lakehouse@toxicitylake7032.dfs.core.windows.net/gold_live/bluesky_scored/


# Phase 2 – Live Toxicity Monitoring on Bluesky

This notebook implements the Phase 2 pipeline:

1. Ingest live posts from Bluesky API into the **Bronze (live)** layer.
2. Clean and standardize them into the **Silver (live)** layer.
3. Engineer features (length + sentiment) into the **Gold (live)** layer.

In [0]:
spark.conf.set(
    "fs.azure.account.key.toxicitylake7032.dfs.core.windows.net",
    "whew that was a close one"
)

In [0]:
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

from datetime import datetime

from atproto import Client
from atproto_client.models import AppBskyFeedGetTimeline

from textblob import TextBlob


# Define Lakehouse paths for the live Bluesky pipeline
BRONZE_PATH = "abfss://lakehouse@toxicitylake7032.dfs.core.windows.net/bronze_live/bluesky/"
SILVER_PATH = "abfss://lakehouse@toxicitylake7032.dfs.core.windows.net/silver_live/bluesky/"
GOLD_PATH   = "abfss://lakehouse@toxicitylake7032.dfs.core.windows.net/gold_live/bluesky_features/"

LIMIT = 50  # how many posts to pull per run (micro-batch size)



In [0]:

# Read credentials from Databricks secrets (never hardcode)
bsky_id = dbutils.secrets.get("bsky-scope", "BSKY_ID").strip()
bsky_pw = dbutils.secrets.get("bsky-scope", "BSKY_APP_PASSWORD").strip()

client = Client()
profile = client.login(bsky_id, bsky_pw)

print("Logged in as:", profile.handle)

Logged in as: [REDACTED]


In [0]:
# Fetch latest posts from our Bluesky home timeline
params = AppBskyFeedGetTimeline.Params(limit=LIMIT)
timeline = client.app.bsky.feed.get_timeline(params=params)

# Normalize feed into a list of items (dict vs object safe)
try:
    feed_items = timeline["feed"]  # dict-like response
except TypeError:
    feed_items = timeline.feed     # attribute-based response

print("Items in feed:", len(feed_items))


Items in feed: 50


In [0]:
rows = []

for item in feed_items:
    # item can be dict or object
    post = item["post"] if isinstance(item, dict) else item.post

    # record + author can also be dict or object
    record = post["record"] if isinstance(post, dict) else post.record
    author = post["author"] if isinstance(post, dict) else post.author

    # Text + created_at
    if isinstance(record, dict):
        text = record.get("text", "")
        created_at = record.get("createdAt")
    else:
        text = getattr(record, "text", "")
        created_at = getattr(record, "created_at", None)

    # Publish timestamp as ISO string
    if isinstance(created_at, str):
        ts = created_at
    elif created_at is not None:
        ts = created_at.isoformat()
    else:
        ts = datetime.utcnow().isoformat()

    # Author info
    if isinstance(author, dict):
        did = author.get("did")
        handle = author.get("handle")
    else:
        did = getattr(author, "did", None)
        handle = getattr(author, "handle", None)

    uri = post["uri"] if isinstance(post, dict) else post.uri

    rows.append(Row(
        post_id = uri,
        user_id = did,
        platform = "bluesky",
        community = "timeline",      # home timeline
        interaction_type = "post",
        publish_ts = ts,
        content = text,
        url = f"https://bsky.app/profile/{handle}/post/{uri.split('/')[-1]}"
    ))

print("Rows built:", len(rows))

if rows:
    df_bronze = spark.createDataFrame(rows)

    (
        df_bronze.write
            .mode("append")
            .format("json")    # Bronze = raw JSON
            .save(BRONZE_PATH)
    )

    display(df_bronze.limit(5))
else:
    print("No posts to write.")


Rows built: 50


post_id,user_id,platform,community,interaction_type,publish_ts,content,url
at://did:plc:keeowbcugv5bsp3hmh6zuszy/app.bsky.feed.post/3m6jdioobcc24,did:plc:keeowbcugv5bsp3hmh6zuszy,bluesky,timeline,post,2025-11-26T07:30:02.101Z,Love watching a documentary where you relate to someone really hard and then they like throw themselves and maybe someone else down a flight of stairs. Metaphorically speaking.,https://bsky.app/profile/bombsfall.bsky.social/post/3m6jdioobcc24
at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6jbnhqmys26,did:plc:cak4klqoj3bqgk5rj6b4f5do,bluesky,timeline,post,2025-11-26T06:56:55.127Z,"In case you were waiting for @joshjohnsoncomedy.bsky.social's take on the Zohran/Trump meeting, wait no longer: www.youtube.com/watch?v=Pxi1...",https://bsky.app/profile/mmasnick.bsky.social/post/3m6jbnhqmys26
at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6jbl4d7j226,did:plc:cak4klqoj3bqgk5rj6b4f5do,bluesky,timeline,post,2025-11-26T06:55:36.045Z,Haha. I did have to pause it to explain that part.,https://bsky.app/profile/mmasnick.bsky.social/post/3m6jbl4d7j226
at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6jae3nss226,did:plc:cak4klqoj3bqgk5rj6b4f5do,bluesky,timeline,post,2025-11-26T06:33:46.721Z,"Oh wow! During that part I was like ""damn, that guy sounds a lot like James Earl Jones, and even looks a little like him, but I don't think that's him...""",https://bsky.app/profile/mmasnick.bsky.social/post/3m6jae3nss226
at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6j6ny6qgc26,did:plc:cak4klqoj3bqgk5rj6b4f5do,bluesky,timeline,post,2025-11-26T06:03:31.142Z,Oh sure.,https://bsky.app/profile/mmasnick.bsky.social/post/3m6j6ny6qgc26


In [0]:
# Read everything from Bronze Live
df_bronze_all = spark.read.json(BRONZE_PATH)

print("Bronze schema:")
df_bronze_all.printSchema()

display(df_bronze_all.limit(5))


Bronze schema:
root
 |-- community: string (nullable = true)
 |-- content: string (nullable = true)
 |-- interaction_type: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- post_id: string (nullable = true)
 |-- publish_ts: string (nullable = true)
 |-- url: string (nullable = true)
 |-- user_id: string (nullable = true)



community,content,interaction_type,platform,post_id,publish_ts,url,user_id
timeline,This got me thinking….how do we as a society normalize pumpkin pie as a breakfast option at restaurants year round? Who will be the change we want to see in the world?,post,bluesky,at://did:plc:2buz4gf5sew3rdwzbxsvcd4s/app.bsky.feed.post/3m6ii7dfboc2q,2025-11-25T23:21:37.253Z,https://bsky.app/profile/minakimes.bsky.social/post/3m6ii7dfboc2q,did:plc:2buz4gf5sew3rdwzbxsvcd4s
timeline,"Not really, no.",post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6iiu53gmk2y,2025-11-25T23:33:15.283Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6iiu53gmk2y,did:plc:cak4klqoj3bqgk5rj6b4f5do
timeline,We do it without spices. It's good.,post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6iitdl2ls2y,2025-11-25T23:32:48.531Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6iitdl2ls2y,did:plc:cak4klqoj3bqgk5rj6b4f5do
timeline,"It's literally not ""same difference."" It's a wholly different thing, which is a key point that I made in discussing it. Also, as I said, what I was calling out was categorically different than what Media Matters did.",post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6iirkfgqk2y,2025-11-25T23:31:48.579Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6iirkfgqk2y,did:plc:cak4klqoj3bqgk5rj6b4f5do
timeline,"Y’all, this CSWEP newsletter is a banger. Five women economists generously share their stories about navigating family and career. 🔥 @jialanw.bsky.social @kmpjones.bsky.social Sarah Hamersma, Kosali Simon, and Sarah Baird. It was an honor to ""edit"" (which mostly involved sitting back for good reads)",post,bluesky,at://did:plc:wclenoal5sj236tmfd5h34p4/app.bsky.feed.post/3m6iatnuxss24,2025-11-25T21:09:49.408Z,https://bsky.app/profile/caitlinmyers.bsky.social/post/3m6iatnuxss24,did:plc:wclenoal5sj236tmfd5h34p4


In [0]:
# Basic cleaning → Silver Live

df_silver = (
    df_bronze_all
    # Drop rows with null or empty content
    .filter(F.col("content").isNotNull() & (F.length(F.col("content")) > 0))
    # Cast publish_ts to proper timestamp
    .withColumn("publish_ts", F.to_timestamp("publish_ts"))
    # Also create a date-only column for easier grouping later
    .withColumn("publish_date_only", F.to_date("publish_ts"))
)

print("Silver row count:", df_silver.count())
df_silver.printSchema()
display(df_silver.limit(5))

# Write Silver as Parquet
(
    df_silver.write
        .mode("overwrite")   # or "append" depending on your strategy
        .format("parquet")
        .save(SILVER_PATH)
)


IOStream.flush timed out


Silver row count: 49
root
 |-- community: string (nullable = true)
 |-- content: string (nullable = true)
 |-- interaction_type: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- post_id: string (nullable = true)
 |-- publish_ts: timestamp (nullable = true)
 |-- url: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- publish_date_only: date (nullable = true)



community,content,interaction_type,platform,post_id,publish_ts,url,user_id,publish_date_only
timeline,This got me thinking….how do we as a society normalize pumpkin pie as a breakfast option at restaurants year round? Who will be the change we want to see in the world?,post,bluesky,at://did:plc:2buz4gf5sew3rdwzbxsvcd4s/app.bsky.feed.post/3m6ii7dfboc2q,2025-11-25T23:21:37.253Z,https://bsky.app/profile/minakimes.bsky.social/post/3m6ii7dfboc2q,did:plc:2buz4gf5sew3rdwzbxsvcd4s,2025-11-25
timeline,"Not really, no.",post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6iiu53gmk2y,2025-11-25T23:33:15.283Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6iiu53gmk2y,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-25
timeline,We do it without spices. It's good.,post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6iitdl2ls2y,2025-11-25T23:32:48.531Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6iitdl2ls2y,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-25
timeline,"It's literally not ""same difference."" It's a wholly different thing, which is a key point that I made in discussing it. Also, as I said, what I was calling out was categorically different than what Media Matters did.",post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6iirkfgqk2y,2025-11-25T23:31:48.579Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6iirkfgqk2y,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-25
timeline,"Y’all, this CSWEP newsletter is a banger. Five women economists generously share their stories about navigating family and career. 🔥 @jialanw.bsky.social @kmpjones.bsky.social Sarah Hamersma, Kosali Simon, and Sarah Baird. It was an honor to ""edit"" (which mostly involved sitting back for good reads)",post,bluesky,at://did:plc:wclenoal5sj236tmfd5h34p4/app.bsky.feed.post/3m6iatnuxss24,2025-11-25T21:09:49.408Z,https://bsky.app/profile/caitlinmyers.bsky.social/post/3m6iatnuxss24,did:plc:wclenoal5sj236tmfd5h34p4,2025-11-25


In [0]:
df_silver_live = spark.read.parquet(SILVER_PATH)

print("Silver Live schema:")
df_silver_live.printSchema()
display(df_silver_live.limit(5))


Silver Live schema:
root
 |-- community: string (nullable = true)
 |-- content: string (nullable = true)
 |-- interaction_type: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- post_id: string (nullable = true)
 |-- publish_ts: timestamp (nullable = true)
 |-- url: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- publish_date_only: date (nullable = true)



community,content,interaction_type,platform,post_id,publish_ts,url,user_id,publish_date_only
timeline,"I thought about that, but figured it was best not to reference. I watched that act on repeat as a kid.",post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6im67acwc27,2025-11-26T00:32:34.309Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6im67acwc27,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-26
timeline,we deliberately wanted to shadow drop so that we can grow the community slowly and prepare for our bigger launches and pushes. So grateful to everyone and sorry in advance that I'm going to be non stop talking about this game,post,bluesky,at://did:plc:4fr4qoylmhzvajcsla5pjbna/app.bsky.feed.post/3m6id6abbzc2u,2025-11-25T21:51:31.714Z,https://bsky.app/profile/seemo.bsky.social/post/3m6id6abbzc2u,did:plc:4fr4qoylmhzvajcsla5pjbna,2025-11-25
timeline,also we're self published and marketing Young Suns completely ourselves (for now) so all your shares mean the world to us <3,post,bluesky,at://did:plc:4fr4qoylmhzvajcsla5pjbna/app.bsky.feed.post/3m6id54muu22u,2025-11-25T21:50:54.345Z,https://bsky.app/profile/seemo.bsky.social/post/3m6id54muu22u,did:plc:4fr4qoylmhzvajcsla5pjbna,2025-11-25
timeline,I will go to my grave insisting that those two things taste the same in their pie forms (and I say that as someone who literally has 12 lbs of sweet potatoes in the trunk of my car right now).,post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6ij3sejjc2y,2025-11-25T23:37:32.482Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6ij3sejjc2y,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-25
timeline,"Generally overrated. Yes, it can make apple pie better, but honestly, just eat ice cream by itself.",post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6iizdd44c2y,2025-11-25T23:36:09.598Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6iizdd44c2y,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-25


In [0]:
# UDFs for TextBlob sentiment and subjectivity
def tb_polarity(text):
    if text is None:
        return 0.0
    try:
        return float(TextBlob(text).sentiment.polarity)
    except Exception:
        return 0.0

def tb_subjectivity(text):
    if text is None:
        return 0.0
    try:
        return float(TextBlob(text).sentiment.subjectivity)
    except Exception:
        return 0.0

udf_sentiment_tb = F.udf(tb_polarity)
udf_subjectivity_tb = F.udf(tb_subjectivity)


In [0]:
df_gold_live = (
    df_silver_live
    # Length features
    .withColumn("content_length_chars", F.length(F.col("content")))
    .withColumn(
        "content_length_words",
        F.size(F.split(F.col("content"), r"\s+"))
    )
    # TextBlob sentiment features
    .withColumn("sentiment_textblob", udf_sentiment_tb(F.col("content")))
    .withColumn("subjectivity_textblob", udf_subjectivity_tb(F.col("content")))
    # 👇 NEW: fake VADER as same as TextBlob for Bluesky
    .withColumn("sentiment_vader", F.col("sentiment_textblob"))
)

print("Gold Live schema:")
df_gold_live.printSchema()
display(df_gold_live.limit(5))

(
    df_gold_live.write
        .mode("overwrite")   # or append
        .format("parquet")
        .save(GOLD_PATH)
)


Gold Live schema:
root
 |-- community: string (nullable = true)
 |-- content: string (nullable = true)
 |-- interaction_type: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- post_id: string (nullable = true)
 |-- publish_ts: timestamp (nullable = true)
 |-- url: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- publish_date_only: date (nullable = true)
 |-- content_length_chars: integer (nullable = true)
 |-- content_length_words: integer (nullable = false)
 |-- sentiment_textblob: string (nullable = true)
 |-- subjectivity_textblob: string (nullable = true)
 |-- sentiment_vader: string (nullable = true)



community,content,interaction_type,platform,post_id,publish_ts,url,user_id,publish_date_only,content_length_chars,content_length_words,sentiment_textblob,subjectivity_textblob,sentiment_vader
timeline,"I thought about that, but figured it was best not to reference. I watched that act on repeat as a kid.",post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6im67acwc27,2025-11-26T00:32:34.309Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6im67acwc27,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-26,102,21,1.0,0.3,1.0
timeline,we deliberately wanted to shadow drop so that we can grow the community slowly and prepare for our bigger launches and pushes. So grateful to everyone and sorry in advance that I'm going to be non stop talking about this game,post,bluesky,at://did:plc:4fr4qoylmhzvajcsla5pjbna/app.bsky.feed.post/3m6id6abbzc2u,2025-11-25T21:51:31.714Z,https://bsky.app/profile/seemo.bsky.social/post/3m6id6abbzc2u,did:plc:4fr4qoylmhzvajcsla5pjbna,2025-11-25,225,41,-0.30000000000000004,0.575,-0.30000000000000004
timeline,also we're self published and marketing Young Suns completely ourselves (for now) so all your shares mean the world to us <3,post,bluesky,at://did:plc:4fr4qoylmhzvajcsla5pjbna/app.bsky.feed.post/3m6id54muu22u,2025-11-25T21:50:54.345Z,https://bsky.app/profile/seemo.bsky.social/post/3m6id54muu22u,did:plc:4fr4qoylmhzvajcsla5pjbna,2025-11-25,124,22,0.221875,0.621875,0.221875
timeline,I will go to my grave insisting that those two things taste the same in their pie forms (and I say that as someone who literally has 12 lbs of sweet potatoes in the trunk of my car right now).,post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6ij3sejjc2y,2025-11-25T23:37:32.482Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6ij3sejjc2y,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-25,192,40,0.2119047619047619,0.4369047619047619,0.2119047619047619
timeline,"Generally overrated. Yes, it can make apple pie better, but honestly, just eat ice cream by itself.",post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6iizdd44c2y,2025-11-25T23:36:09.598Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6iizdd44c2y,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-25,99,17,0.3833333333333333,0.6333333333333333,0.3833333333333333


## Apply Reddit Toxicity Model to Live Bluesky Data

In this section we:
1. Load the **Gold Live** Bluesky features.
2. Load our previously trained **Reddit toxicity model** from MLflow.
3. Run predictions on Bluesky posts.
4. Save a scored version of the dataset for dashboards / analysis.


In [0]:
# Reload Gold Live Bluesky features (just to be explicit)
df_gold_live = spark.read.parquet(GOLD_PATH)

print("Gold Live (Bluesky) schema:")
df_gold_live.printSchema()
display(df_gold_live.limit(10))


Gold Live (Bluesky) schema:
root
 |-- community: string (nullable = true)
 |-- content: string (nullable = true)
 |-- interaction_type: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- post_id: string (nullable = true)
 |-- publish_ts: timestamp (nullable = true)
 |-- url: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- publish_date_only: date (nullable = true)
 |-- content_length_chars: integer (nullable = true)
 |-- content_length_words: integer (nullable = true)
 |-- sentiment_textblob: string (nullable = true)
 |-- subjectivity_textblob: string (nullable = true)
 |-- sentiment_vader: string (nullable = true)



community,content,interaction_type,platform,post_id,publish_ts,url,user_id,publish_date_only,content_length_chars,content_length_words,sentiment_textblob,subjectivity_textblob,sentiment_vader
timeline,"I thought about that, but figured it was best not to reference. I watched that act on repeat as a kid.",post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6im67acwc27,2025-11-26T00:32:34.309Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6im67acwc27,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-26,102,21,1.0,0.3,1.0
timeline,we deliberately wanted to shadow drop so that we can grow the community slowly and prepare for our bigger launches and pushes. So grateful to everyone and sorry in advance that I'm going to be non stop talking about this game,post,bluesky,at://did:plc:4fr4qoylmhzvajcsla5pjbna/app.bsky.feed.post/3m6id6abbzc2u,2025-11-25T21:51:31.714Z,https://bsky.app/profile/seemo.bsky.social/post/3m6id6abbzc2u,did:plc:4fr4qoylmhzvajcsla5pjbna,2025-11-25,225,41,-0.30000000000000004,0.575,-0.30000000000000004
timeline,also we're self published and marketing Young Suns completely ourselves (for now) so all your shares mean the world to us <3,post,bluesky,at://did:plc:4fr4qoylmhzvajcsla5pjbna/app.bsky.feed.post/3m6id54muu22u,2025-11-25T21:50:54.345Z,https://bsky.app/profile/seemo.bsky.social/post/3m6id54muu22u,did:plc:4fr4qoylmhzvajcsla5pjbna,2025-11-25,124,22,0.221875,0.621875,0.221875
timeline,I will go to my grave insisting that those two things taste the same in their pie forms (and I say that as someone who literally has 12 lbs of sweet potatoes in the trunk of my car right now).,post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6ij3sejjc2y,2025-11-25T23:37:32.482Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6ij3sejjc2y,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-25,192,40,0.2119047619047619,0.4369047619047619,0.2119047619047619
timeline,"Generally overrated. Yes, it can make apple pie better, but honestly, just eat ice cream by itself.",post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6iizdd44c2y,2025-11-25T23:36:09.598Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6iizdd44c2y,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-25,99,17,0.3833333333333333,0.6333333333333333,0.3833333333333333
timeline,"FWIW, it is my wife who has convinced my whole family that the pumpkin pie she makes (very low sugar content, minimal spices) is good breakfast food. She also has a low sugar chocolate cake that she is convinced is breakfast food, and my kids are wholly on board with this.",post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6iixn7a6s2y,2025-11-25T23:35:12.848Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6iixn7a6s2y,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-25,273,51,0.15999999999999998,0.458,0.15999999999999998
timeline,This got me thinking….how do we as a society normalize pumpkin pie as a breakfast option at restaurants year round? Who will be the change we want to see in the world?,post,bluesky,at://did:plc:2buz4gf5sew3rdwzbxsvcd4s/app.bsky.feed.post/3m6ii7dfboc2q,2025-11-25T23:21:37.253Z,https://bsky.app/profile/minakimes.bsky.social/post/3m6ii7dfboc2q,did:plc:2buz4gf5sew3rdwzbxsvcd4s,2025-11-25,167,32,-0.2,0.4,-0.2
timeline,"Not really, no.",post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6iiu53gmk2y,2025-11-25T23:33:15.283Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6iiu53gmk2y,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-25,15,3,-0.1,0.2,-0.1
timeline,We do it without spices. It's good.,post,bluesky,at://did:plc:cak4klqoj3bqgk5rj6b4f5do/app.bsky.feed.post/3m6iitdl2ls2y,2025-11-25T23:32:48.531Z,https://bsky.app/profile/mmasnick.bsky.social/post/3m6iitdl2ls2y,did:plc:cak4klqoj3bqgk5rj6b4f5do,2025-11-25,35,7,0.7,0.6000000000000001,0.7
timeline,"It's literally not ""same difference."" It's a wholly different thing, which is a key point that I made in discussing it. Also, as I said, what I was c

In [0]:
# Take a sample of Gold Live data for scoring
df_gold_live_sample = df_gold_live.limit(200)

# Convert to pandas because our model is a scikit-learn pipeline
pdf_live = df_gold_live_sample.toPandas()
print("Bluesky sample shape:", pdf_live.shape)

# Feature columns the model expects
feature_cols = [
    "content",
    "content_length_words",
    "content_length_chars",
    "sentiment_textblob",
    "subjectivity_textblob",
    "sentiment_vader"
]

X_live = pdf_live[feature_cols]


Bluesky sample shape: (49, 14)


In [0]:
import mlflow
import mlflow.sklearn

# 👇 Replace this with the actual run ID from your Reddit training
RUN_ID = "blah"

# This assumes you logged the model as:
# mlflow.sklearn.log_model(model, artifact_path="model")
logged_model_uri = f"runs:/{RUN_ID}/model"

print("Loading model from:", logged_model_uri)
reddit_toxicity_model = mlflow.sklearn.load_model(logged_model_uri)


Loading model from: runs:/a0d07d615b264f39ad0a8a2a91a06e7f/model


In [0]:
# Predict toxicity on the Bluesky sample
y_pred = reddit_toxicity_model.predict(X_live)

# If it's a probabilistic classifier (e.g., LogisticRegression), also get probability
try:
    y_proba = reddit_toxicity_model.predict_proba(X_live)[:, 1]
except Exception:
    y_proba = None

# Attach predictions to the pandas DataFrame
pdf_live["predicted_toxicity_label"] = y_pred

if y_proba is not None:
    pdf_live["predicted_toxicity_score"] = y_proba

# Show a few scored posts
pdf_live[["content", "predicted_toxicity_label"] + (
    ["predicted_toxicity_score"] if y_proba is not None else []
)]


,content,predicted_toxicity_label,predicted_toxicity_score
0,Twitter accounts are based in Russia. BlueSky ...,0,0.104499
1,I’m still thinking about Bari Weiss saying tha...,0,0.091265
2,"""A woman with a family connection to White Hou...",0,0.074910
3,The first official confirmation that Noem hers...,0,0.098112
4,when he does the slow blink at someone that me...,0,0.350658
5,"I talk to many members, including my own.",0,0.036858
6,THOSE ARE NOT CONCESSIONS!!,0,0.171681
7,Next week the House E&C Committee is looking t...,0,0.135725
8,"So, a senior Justice Department official — alo...",0,0.050765
9,It’s sad that “soldiers shouldn’t commit crime...,0,0.238894


In [0]:
# Convert back to Spark
df_scored_live = spark.createDataFrame(pdf_live)

display(df_scored_live.select(
    "platform",
    "publish_ts",
    "content",
    "predicted_toxicity_label",
    F.col("predicted_toxicity_score") if "predicted_toxicity_score" in df_scored_live.columns else F.lit(None).alias("predicted_toxicity_score")
).limit(10))


platform,publish_ts,content,predicted_toxicity_label,predicted_toxicity_score
bluesky,2025-11-23T20:29:20.074Z,"Twitter accounts are based in Russia. BlueSky accounts are based in homes with, frankly, too many books, plants, obsolete cables, and pieces of rustic pottery, that could do with a bit of a tidying up, to be honest.",0,0.10449947978163927
bluesky,2025-11-26T02:15:32.346Z,"I’m still thinking about Bari Weiss saying that Hasan Piker (extremely popular influencer) is less of a measure of where Americans are than Alan Dershowitz (extremely unpopular Epstein associate, his own neighbors won’t sell him a pierogi)",0,0.09126540071115036
bluesky,2025-11-26T04:07:37.584Z,"""A woman with a family connection to White House Press Secretary Karoline Leavitt has been arrested by Immigration and Customs Enforcement agents in Revere, Massachusetts."" www.wcvb.com/article/karo...",0,0.07491001193743843
bluesky,2025-11-26T01:45:59.176Z,"The first official confirmation that Noem herself made the decision not to turn the planes around. A couple hundred men were tortured as a result — and soon after, the Supreme Court confirmed that the Trump admin’s Alien Enemies Act process was a total violation of due process.",0,0.09811196922170984
bluesky,2025-11-26T05:08:41.133Z,when he does the slow blink at someone that means he likes them,0,0.35065816913786696
bluesky,2025-11-26T05:11:49.995Z,"I talk to many members, including my own.",0,0.03685786482365462
bluesky,2025-11-26T00:39:15.734Z,THOSE ARE NOT CONCESSIONS!!,0,0.17168135250885377
bluesky,2025-11-26T00:50:15.249Z,"Next week the House E&C Committee is looking to run something of a shock and awe campaign, debating at least 19 bills that they say are related to ""protecting kids online."" Most of which are actually about censorship and control. Don't buy the framing. It's always the same damn thing.",0,0.13572524203187974
bluesky,2025-11-25T23:31:35.67Z,"So, a senior Justice Department official — along with several members of Congress — used the authority of their offices to boost an obviously dubious conspiracy theory accusing a federal employee of a horrific crime. And now it’s clear the allegation was baseless. www.cbsnews.com/news/jan-6-p...",0,0.05076501912442137
bluesky,2025-11-25T23:48:30.68Z,It’s sad that “soldiers shouldn’t commit crimes” is a brave position for a Republican to take.,0,0.23889370307213773


In [0]:
SCORED_PATH = "abfss://lakehouse@toxicitylake7032.dfs.core.windows.net/gold_live/bluesky_scored/"

(
    df_scored_live.write
        .mode("overwrite")   # or "append" if you want to accumulate history
        .format("parquet")
        .save(SCORED_PATH)
)

print("Saved scored Bluesky data to:", SCORED_PATH)


Saved scored Bluesky data to: abfss://lakehouse@toxicitylake7032.dfs.core.windows.net/gold_live/bluesky_scored/


Stuff that im tetsing out down here in a land down under